In [18]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import pandas as pd
import dask.dataframe as dd

device = torch.device("cuda:0")

In [21]:
# Loading CSV

trainingDataCSV = pd.read_csv('evenodd_training.csv')
trainingData = torch.tensor(trainingDataCSV.values)

testDataCSV = pd.read_csv('evenodd.csv')
testData = torch.tensor(testDataCSV.values)


In [25]:
# Seperate into Training and Test data

our_batch_size = 20

train_dataloader = DataLoader(trainingData, batch_size=our_batch_size)
test_dataloader = DataLoader(testData, batch_size=our_batch_size)

print(f"""
    train_dataloader:
    {train_dataloader.dataset}


    test_dataloader:
    {test_dataloader}
""")


for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break



    train_dataloader:
    tensor([[3, 1],
        [4, 0],
        [5, 1],
        [3, 1],
        [4, 0],
        [5, 1],
        [3, 1],
        [4, 0],
        [5, 1],
        [3, 1],
        [4, 0],
        [5, 1],
        [3, 1],
        [4, 0],
        [5, 1],
        [3, 1],
        [4, 0],
        [5, 1],
        [3, 1],
        [4, 0],
        [5, 1],
        [3, 1],
        [4, 0],
        [5, 1],
        [3, 1],
        [4, 0],
        [5, 1],
        [3, 1],
        [4, 0],
        [5, 1],
        [3, 1],
        [4, 0],
        [5, 1],
        [3, 1],
        [4, 0],
        [5, 1],
        [3, 1],
        [4, 0],
        [5, 1],
        [3, 1],
        [4, 0],
        [5, 1],
        [3, 1],
        [4, 0],
        [5, 1],
        [3, 1],
        [4, 0],
        [5, 1],
        [3, 1],
        [4, 0],
        [5, 1],
        [3, 1],
        [4, 0],
        [5, 1],
        [3, 1],
        [4, 0],
        [5, 1],
        [3, 1],
        [4, 0],
        [5, 1],
        [3, 1

ValueError: too many values to unpack (expected 2)

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else 'mps'
    if torch.backends.mps.is_advaliable()
    else "cpu"
)
print(f"Using {device} for data processing")

# Define Model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        return self.linear_relu_stack(x)
        
model = NeuralNetwork().to(device)
print(model)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 2
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

In [ ]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

In [ ]:
classes = [
    "
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')